In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestClassifier
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from utility.sentiment import *
import utility.text_normalizer as tn
import tqdm
import spacy

# Load data

In [32]:
# Load all the manually tagged data
tag1 = pd.read_excel('tagged_tweets/tagged_1.xlsx')
tag2 = pd.read_excel('tagged_tweets/tagged_2.xls')
tag3 = pd.read_excel('tagged_tweets/tagged_3.xls')
tag4 = pd.read_excel('tagged_tweets/tagged_4.xls')
tag5 = pd.read_excel('tagged_tweets/tagged_5.xlsx')

/Users/Isaacbolo/opt/anaconda3/envs/CDFW/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/Isaacbolo/opt/anaconda3/envs/CDFW/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [33]:
tagged_tweets = pd.concat([tag1, tag2, tag3, tag4, tag5])

In [46]:
tagged_tweets.sample(10)

,tweet_id,text,tag
399,3.935967e+17,Coyotes.... http://t.co/2POIPjVqeE,NaN
857,1.901608e+17,Coyotes force closure of trails in GG Park to ...,1.0
606,7.034044e+17,Davis!! (@ Dos Coyotes Border Cafe - @mydoscoy...,0.0
545,2.459792e+17,Trying out this place. (@ Coyote Grill) [pic]:...,0.0
211,5.399246e+16,At Coyote Bar & Grill they call this the 'Bubb...,0.0
456,1.515059e+17,Gooooo Kings!!! (@ STAPLES Center for Phoenix ...,NaN
773,7.886307e+17,There was a coyote running down my street,1.0
882,8.070402e+17,@Twitt_Itches @TheAtlantic need to bring giraf...,1.0
106,7.860966e+17,Tonight I have mostly been hanging out in the ...,0.0
376,9.082721e+16,"I'm at Chase Bank (4571 Los Coyotes Diagonal, ...",0.0


In [53]:
tagged_tweets['tweet_id'] = tagged_tweets['tweet_id'].astype('int').astype('O')

In [54]:
tagged_tweets

,tweet_id,text,tag
0,1215345232016880128,@ABC Coyotes rarely attack people unless they ...,1.0
1,1081351054925609984,Crazy girl caught a cricket earlier 😅🤦🏽‍♂️ @ C...,0.0
2,1195543563838220032,Division 1 playoffs Calabasas vs Saint John Bo...,0.0
3,1207791734316389888,Someone got hold of video of @Doc_Coyote meeti...,0.0
4,883523800725240960,Name of that drink: Cranky coyote @ Ace Hotel ...,0.0
...,...,...,...
995,12961025407,I just became the mayor of Coyote Taqueria on ...,0.0
996,1319463344005279488,You stupid stupid jackasses. These children ha...,0.0
997,1315857557802540032,This is the old barn at P Ranch on Malhuer NWF...,1.0
998,836671614888497024,New web tool tracks #coyote activity using sha...,1.0


Load the complete dataset

In [89]:
data = pd.read_parquet('modeling_notebooks/data/all_tweets.gzip')
data

,grid_id,tweet_id,created_at,text,author_id,place_id,lat,long,full_name,name,place_type,id,county
0,1,None,NaT,None,None,None,NaN,NaN,None,None,None,1,DEL NORTE
1,2,None,NaT,None,None,None,NaN,NaN,None,None,None,2,DEL NORTE
2,3,None,NaT,None,None,None,NaN,NaN,None,None,None,3,DEL NORTE
3,4,None,NaT,None,None,None,NaN,NaN,None,None,None,4,DEL NORTE
4,5,None,NaT,None,None,None,NaN,NaN,None,None,None,5,DEL NORTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48590,4087,None,NaT,None,None,None,NaN,NaN,None,None,None,4087,SAN DIEGO
48591,4088,699642589348306945,2016-02-16 17:12:46,One of my favorite short #roadbike rides with ...,236074117,fbd6d2f5a4e4a15e,-116.612000,32.604700,"California, USA",California,admin,4088,SAN DIEGO
48592,4088,1149699890500591616,2019-07-12 15:19:48,Molly in her coyote vest decided she wanted to...,167237995,fbd6d2f5a4e4a15e,-116.592854,32.612469,"California, USA",California,admin,4088,SAN DIEGO
48593,4089,828666935525199874,2017-02-06 18:09:27,DOP for Undead Express. The suns down and the ...,17798317,fbd6d2f5a4e4a15e,-116.472193,32.613785,"California, USA",California,admin,4089,SAN DIEGO


Join datasets on text

In [90]:
data = data.merge(tagged_tweets, how='left', on='text')

In [91]:
data

,grid_id,tweet_id_x,created_at,text,author_id,place_id,lat,long,full_name,name,place_type,id,county,tweet_id_y,tag
0,1,None,NaT,None,None,None,NaN,NaN,None,None,None,1,DEL NORTE,NaN,NaN
1,2,None,NaT,None,None,None,NaN,NaN,None,None,None,2,DEL NORTE,NaN,NaN
2,3,None,NaT,None,None,None,NaN,NaN,None,None,None,3,DEL NORTE,NaN,NaN
3,4,None,NaT,None,None,None,NaN,NaN,None,None,None,4,DEL NORTE,NaN,NaN
4,5,None,NaT,None,None,None,NaN,NaN,None,None,None,5,DEL NORTE,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48876,4087,None,NaT,None,None,None,NaN,NaN,None,None,None,4087,SAN DIEGO,NaN,NaN
48877,4088,699642589348306945,2016-02-16 17:12:46,One of my favorite short #roadbike rides with ...,236074117,fbd6d2f5a4e4a15e,-116.612000,32.604700,"California, USA",California,admin,4088,SAN DIEGO,NaN,NaN
48878,4088,1149699890500591616,2019-07-12 15:19:48,Molly in her coyote vest decided she wanted to...,167237995,fbd6d2f5a4e4a15e,-116.592854,32.612469,"California, USA",California,admin,4088,SAN DIEGO,NaN,NaN
48879,4089,828666935525199874,2017-02-06 18:09:27,DOP for Undead Express. The suns down and the ...,17798317,fbd6d2f5a4e4a15e,-116.472193,32.613785,"California, USA",California,admin,4089,SAN DIEGO,NaN,NaN


In [92]:
data['tag'].notnull().sum()

4648

In [94]:
# Get rows with text
notnull_data = data.loc[data['text'].notnull()]

In [95]:
notnull_data = notnull_data.reset_index(drop=True)
notnull_data

,grid_id,tweet_id_x,created_at,text,author_id,place_id,lat,long,full_name,name,place_type,id,county,tweet_id_y,tag
0,40,820841383099437058,2017-01-16 03:53:30,Back of the camera preview from today at the r...,231610931,fbd6d2f5a4e4a15e,-121.697000,41.948800,"California, USA",California,admin,40,SISKIYOU,820841383099436928,1.0
1,40,820841383099437058,2017-01-16 03:53:30,Back of the camera preview from today at the r...,231610931,fbd6d2f5a4e4a15e,-121.697000,41.948800,"California, USA",California,admin,40,SISKIYOU,820841383099437056,1.0
2,67,310585251560841216,2013-03-10 02:57:49,@phoenixcoyotes. Nicccce starting the 3rd with...,69728061,fbd6d2f5a4e4a15e,-122.581384,41.805792,"California, USA",California,admin,67,SISKIYOU,310585251560841216,0.0
3,73,19057484971,2010-07-21 06:52:06,And now I hear coyotes... they better not hurt...,54453264,ca06c9bd1e591f53,NaN,NaN,"Butte Valley, CA",Butte Valley,city,73,SISKIYOU,19057484971,1.0
4,101,1115718553406033921,2019-04-09 20:50:06,@almostjingo Not to mention American citizens ...,823970107,e057ac551ce5df02,NaN,NaN,"Yreka, CA",Yreka,city,101,SISKIYOU,1115718553406033920,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45779,4085,185202748202033152,2012-03-29 03:12:30,Who runs with dogs this late?? Coyotes Still...,375866895,28ace6b8d6dbc3af,-116.945490,32.643177,"Chula Vista, CA",Chula Vista,city,4085,SAN DIEGO,NaN,NaN
45780,4086,1195116824913776640,2019-11-14 23:10:29,"In the 90s, when border enforcement was still ...",15118889,07d9d3592f487000,NaN,NaN,Otay Mountain Truck Trail,Otay Mountain Truck Trail,poi,4086,SAN DIEGO,NaN,NaN
45781,4088,699642589348306945,2016-02-16 17:12:46,One of my favorite short #roadbike rides with ...,236074117,fbd6d2f5a4e4a15e,-116.612000,32.604700,"California, USA",California,admin,4088,SAN DIEGO,NaN,NaN
45782,4088,1149699890500591616,2019-07-12 15:19:48,Molly in her coyote vest decided she wanted to...,167237995,fbd6d2f5a4e4a15e,-116.592854,32.612469,"California, USA",California,admin,4088,SAN DIEGO,NaN,NaN


# Predict: Convert text to word embeddings

In [96]:
def tweet_vector(nlp, tweet):
    tweet = tn.tweet_preprocessor2(tweet)
    tweet = nlp(tweet)
    return tweet.vector


def tweets2vector(nlp, tweets):
    vectors = []
    for tweet in tqdm.tqdm(tweets):
        vectors.append(tweet_vector(nlp, tweet))
    return np.array(vectors)

In [97]:
nlp = spacy.load('en_core_web_md')

In [98]:
word_embeddings = tweets2vector(nlp, notnull_data['text'])

100%|██████████| 45784/45784 [04:01<00:00, 189.74it/s]


In [99]:
word_embeddings

array([[-0.00965526,  0.09418506, -0.162947  , ...,  0.04442416,
        -0.03511579,  0.13016784],
       [-0.00965526,  0.09418506, -0.162947  , ...,  0.04442416,
        -0.03511579,  0.13016784],
       [-0.04693058,  0.14849557, -0.05399417, ..., -0.05143673,
        -0.02890075,  0.02756908],
       ...,
       [ 0.1400825 ,  0.08723924, -0.2610565 , ..., -0.03904875,
        -0.01670982, -0.02110842],
       [-0.06393279,  0.15761217, -0.02621064, ..., -0.00551164,
         0.07185886,  0.12273428],
       [ 0.06804594, -0.00928667, -0.07735228, ..., -0.00643467,
        -0.03804067,  0.02941406]], dtype=float32)

Load model and predict on word embeddings

In [100]:
model = joblib.load('../../models/xgc_.89.pkl')

In [101]:
preds = model.predict(word_embeddings)

In [102]:
preds

array([1., 1., 0., ..., 0., 1., 0.])

In [103]:
notnull_data['relevance_pred'] = preds

In [105]:
notnull_data[['text', 'tag', 'relevance_pred']]

,text,tag,relevance_pred
0,Back of the camera preview from today at the r...,1.0,1.0
1,Back of the camera preview from today at the r...,1.0,1.0
2,@phoenixcoyotes. Nicccce starting the 3rd with...,0.0,0.0
3,And now I hear coyotes... they better not hurt...,1.0,1.0
4,@almostjingo Not to mention American citizens ...,0.0,0.0
...,...,...,...
45779,Who runs with dogs this late?? Coyotes Still...,NaN,1.0
45780,"In the 90s, when border enforcement was still ...",NaN,0.0
45781,One of my favorite short #roadbike rides with ...,NaN,0.0
45782,Molly in her coyote vest decided she wanted to...,NaN,1.0


Get sentiment score of relevant tweets

In [106]:
rel_tweets = notnull_data[(notnull_data['tag'] == 1) | (notnull_data['relevance_pred'] == 1)]
rel_tweets.shape

(14838, 16)

In [107]:
rel_tweets

,grid_id,tweet_id_x,created_at,text,author_id,place_id,lat,long,full_name,name,place_type,id,county,tweet_id_y,tag,relevance_pred
0,40,820841383099437058,2017-01-16 03:53:30,Back of the camera preview from today at the r...,231610931,fbd6d2f5a4e4a15e,-121.697000,41.948800,"California, USA",California,admin,40,SISKIYOU,820841383099436928,1.0,1.0
1,40,820841383099437058,2017-01-16 03:53:30,Back of the camera preview from today at the r...,231610931,fbd6d2f5a4e4a15e,-121.697000,41.948800,"California, USA",California,admin,40,SISKIYOU,820841383099437056,1.0,1.0
3,73,19057484971,2010-07-21 06:52:06,And now I hear coyotes... they better not hurt...,54453264,ca06c9bd1e591f53,NaN,NaN,"Butte Valley, CA",Butte Valley,city,73,SISKIYOU,19057484971,1.0,1.0
5,101,364985527600033795,2013-08-07 05:45:05,Goal for tomorrow is to rope a coyote #gottah...,266238333,e057ac551ce5df02,-122.611145,41.742534,"Yreka, CA",Yreka,city,101,SISKIYOU,364985527600033792,1.0,0.0
6,101,1397357765811597312,2021-05-26 01:03:47,@numinousstock I could potentially run into a ...,2802522932,e057ac551ce5df02,NaN,NaN,"Yreka, CA",Yreka,city,101,SISKIYOU,1397357765811597312,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45776,4085,212411891711090689,2012-06-12 05:11:56,"Just saw a coyote, thought it was a teen wolf ...",449271004,28ace6b8d6dbc3af,-116.950282,32.658601,"Chula Vista, CA",Chula Vista,city,4085,SAN DIEGO,NaN,NaN,1.0
45777,4085,210056659815501825,2012-06-05 17:13:05,Coyotes refuel. @ New Hope Community Church ...,457642307,28ace6b8d6dbc3af,-116.940160,32.631622,"Chula Vista, CA",Chula Vista,city,4085,SAN DIEGO,NaN,NaN,1.0
45778,4085,699096334415089664,2016-02-15 05:02:09,Listening to the howling of coyotes. @thousan...,11954102,fbd6d2f5a4e4a15e,-116.852493,32.649029,"California, USA",California,admin,4085,SAN DIEGO,699096334415089664,1.0,1.0
45779,4085,185202748202033152,2012-03-29 03:12:30,Who runs with dogs this late?? Coyotes Still...,375866895,28ace6b8d6dbc3af,-116.945490,32.643177,"Chula Vista, CA",Chula Vista,city,4085,SAN DIEGO,NaN,NaN,1.0


In [108]:
sent_scores = [score_tweet(t) for t in rel_tweets['text']]

In [110]:
df_sent = pd.DataFrame.from_records(sent_scores)
df_sent

,neg_sent_score,neu_sent_score,pos_sent_score,compound_sent_score,sent_class
0,0.094,0.906,0.000,-0.2411,negative
1,0.094,0.906,0.000,-0.2411,negative
2,0.000,0.579,0.421,0.7616,positive
3,0.000,1.000,0.000,0.0000,neutral
4,0.000,1.000,0.000,0.0000,neutral
...,...,...,...,...,...
14833,0.000,1.000,0.000,0.0000,neutral
14834,0.000,0.707,0.293,0.4404,positive
14835,0.000,1.000,0.000,0.0000,neutral
14836,0.000,1.000,0.000,0.0000,neutral


In [111]:
rel_tweets = pd.concat([rel_tweets, df_sent], axis=1)

In [112]:
rel_tweets

,grid_id,tweet_id_x,created_at,text,author_id,place_id,lat,long,full_name,name,...,id,county,tweet_id_y,tag,relevance_pred,neg_sent_score,neu_sent_score,pos_sent_score,compound_sent_score,sent_class
0,40.0,820841383099437058,2017-01-16 03:53:30,Back of the camera preview from today at the r...,231610931,fbd6d2f5a4e4a15e,-121.697000,41.948800,"California, USA",California,...,40.0,SISKIYOU,820841383099436928,1.0,1.0,0.094,0.906,0.000,-0.2411,negative
1,40.0,820841383099437058,2017-01-16 03:53:30,Back of the camera preview from today at the r...,231610931,fbd6d2f5a4e4a15e,-121.697000,41.948800,"California, USA",California,...,40.0,SISKIYOU,820841383099437056,1.0,1.0,0.094,0.906,0.000,-0.2411,negative
3,73.0,19057484971,2010-07-21 06:52:06,And now I hear coyotes... they better not hurt...,54453264,ca06c9bd1e591f53,NaN,NaN,"Butte Valley, CA",Butte Valley,...,73.0,SISKIYOU,19057484971,1.0,1.0,0.000,1.000,0.000,0.0000,neutral
5,101.0,364985527600033795,2013-08-07 05:45:05,Goal for tomorrow is to rope a coyote #gottah...,266238333,e057ac551ce5df02,-122.611145,41.742534,"Yreka, CA",Yreka,...,101.0,SISKIYOU,364985527600033792,1.0,0.0,0.000,0.955,0.045,0.2732,positive
6,101.0,1397357765811597312,2021-05-26 01:03:47,@numinousstock I could potentially run into a ...,2802522932,e057ac551ce5df02,NaN,NaN,"Yreka, CA",Yreka,...,101.0,SISKIYOU,1397357765811597312,1.0,1.0,0.000,1.000,0.000,0.0000,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14830,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000,0.607,0.393,0.7974,positive
14832,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000,1.000,0.000,0.0000,neutral
14834,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000,0.707,0.293,0.4404,positive
14836,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000,1.000,0.000,0.0000,neutral


Save relevant tweets with sentiment scores

In [113]:
# rel_tweets.to_parquet('modeling_notebooks/data/rel_tweets_with_sentiment.gzip', index=None)

In [114]:
data = pd.read_parquet('modeling_notebooks/data/rel_tweets_with_sentiment.gzip')
data.head()

,grid_id,tweet_id_x,created_at,text,author_id,place_id,lat,long,full_name,name,...,id,county,tweet_id_y,tag,relevance_pred,neg_sent_score,neu_sent_score,pos_sent_score,compound_sent_score,sent_class
0,40.0,820841383099437058,2017-01-16 03:53:30,Back of the camera preview from today at the r...,231610931,fbd6d2f5a4e4a15e,-121.697000,41.948800,"California, USA",California,...,40.0,SISKIYOU,8.208414e+17,1.0,1.0,0.094,0.906,0.000,-0.2411,negative
1,40.0,820841383099437058,2017-01-16 03:53:30,Back of the camera preview from today at the r...,231610931,fbd6d2f5a4e4a15e,-121.697000,41.948800,"California, USA",California,...,40.0,SISKIYOU,8.208414e+17,1.0,1.0,0.094,0.906,0.000,-0.2411,negative
3,73.0,19057484971,2010-07-21 06:52:06,And now I hear coyotes... they better not hurt...,54453264,ca06c9bd1e591f53,NaN,NaN,"Butte Valley, CA",Butte Valley,...,73.0,SISKIYOU,1.905748e+10,1.0,1.0,0.000,1.000,0.000,0.0000,neutral
5,101.0,364985527600033795,2013-08-07 05:45:05,Goal for tomorrow is to rope a coyote #gottah...,266238333,e057ac551ce5df02,-122.611145,41.742534,"Yreka, CA",Yreka,...,101.0,SISKIYOU,3.649855e+17,1.0,0.0,0.000,0.955,0.045,0.2732,positive
6,101.0,1397357765811597312,2021-05-26 01:03:47,@numinousstock I could potentially run into a ...,2802522932,e057ac551ce5df02,NaN,NaN,"Yreka, CA",Yreka,...,101.0,SISKIYOU,1.397358e+18,1.0,1.0,0.000,1.000,0.000,0.0000,neutral


In [115]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25636 entries, 0 to 14837
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   grid_id              14838 non-null  float64       
 1   tweet_id_x           14838 non-null  object        
 2   created_at           14838 non-null  datetime64[ns]
 3   text                 14838 non-null  object        
 4   author_id            14838 non-null  object        
 5   place_id             14722 non-null  object        
 6   lat                  5572 non-null   float64       
 7   long                 5572 non-null   float64       
 8   full_name            14719 non-null  object        
 9   name                 14719 non-null  object        
 10  place_type           14719 non-null  object        
 11  id                   14838 non-null  float64       
 12  county               14838 non-null  object        
 13  tweet_id_y           1552 non-n

# Sentiment Analysis